# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling to prevent OOM errors for large batches. For details on this cache-aware scheduling algorithm, see our [paper](https://arxiv.org/pdf/2312.07104).

In [1]:
# launch the offline engine

import sglang as sgl
from sglang.utils import print_highlight
import asyncio

llm = sgl.Engine(model_path="meta-llama/Meta-Llama-3.1-8B-Instruct")

[2024-11-03 05:05:53] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=True, dtype='auto', kv_cache_dtype='auto', quantization=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3.1-8B-Instruct', chat_template=None, is_embedding=False, host='127.0.0.1', port=30000, mem_fraction_static=0.88, max_running_requests=None, max_total_tokens=None, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, tp_size=1, stream_interval=1, random_seed=711836784, constrained_json_whitespace_pattern=None, decode_log_interval=40, log_level='info', log_level_http=None, log_requests=False, show_time_cost=False, api_key=None, file_storage_pth='SGLang_storage', enable_cache_report=False, watchdog_timeout=600, dp_size=1, load_balance_method='round_robin', dist_

[2024-11-03 05:06:08 TP0] Init torch distributed begin.


[2024-11-03 05:06:09 TP0] Load weight begin. avail mem=78.59 GB


[2024-11-03 05:06:09 TP0] lm_eval is not installed, GPTQ may not be usable


INFO 11-03 05:06:10 weight_utils.py:243] Using model weights format ['*.safetensors']


...

Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:02,  1.23it/s]


Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.12it/s]


Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.11it/s]


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.48it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.33it/s]

[2024-11-03 05:06:13 TP0] Load weight end. type=LlamaForCausalLM, dtype=torch.bfloat16, avail mem=63.50 GB
[2024-11-03 05:06:14 TP0] Memory pool end. avail mem=8.37 GB
[2024-11-03 05:06:14 TP0] Capture cuda graph begin. This can take up to several minutes.


[2024-11-03 05:06:21 TP0] max_total_num_tokens=442913, max_prefill_tokens=16384, max_running_requests=2049, context_len=131072


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print_highlight("===============================")
    print_highlight(f"Prompt: {prompt}\nGenerated text: {output['text']}")

[2024-11-03 05:06:21 TP0] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2024-11-03 05:06:21 TP0] Prefill batch. #new-seq: 3, #new-token: 17, #cached-token: 3, cache hit rate: 11.54%, token usage: 0.00, #running-req: 1, #queue-req: 0


[2024-11-03 05:06:22 TP0] Decode batch. #running-req: 4, #token: 181, token usage: 0.00, gen throughput (token/s): 195.51, #queue-req: 0


[2024-11-03 05:06:22 TP0] Decode batch. #running-req: 4, #token: 341, token usage: 0.00, gen throughput (token/s): 550.13, #queue-req: 0


[2024-11-03 05:06:22 TP0] Decode batch. #running-req: 4, #token: 501, token usage: 0.00, gen throughput (token/s): 545.61, #queue-req: 0


### Streaming Synchronous Generation

In [3]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print_highlight("\n=== Testing synchronous streaming generation ===")

for prompt in prompts:
    print_highlight(f"\nPrompt: {prompt}")
    print("Generated text: ", end="", flush=True)

    for chunk in llm.generate(prompt, sampling_params, stream=True):
        print(chunk["text"], end="", flush=True)
    print()

Generated text: 

 Donna

 and

 I

'm

...

 is

 bias

.

[2024-11-03 05:06:25 TP0] Decode batch. #running-req: 1, #token: 105, token usage: 0.00, gen throughput (token/s): 139.29, #queue-req: 0


 AI

### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print_highlight("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print_highlight(f"\nPrompt: {prompt}")
        print_highlight(f"Generated text: {output['text']}")


asyncio.run(main())

[2024-11-03 05:06:25 TP0] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 5, cache hit rate: 46.00%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2024-11-03 05:06:25 TP0] Prefill batch. #new-seq: 2, #new-token: 2, #cached-token: 10, cache hit rate: 53.23%, token usage: 0.00, #running-req: 1, #queue-req: 0
[2024-11-03 05:06:25 TP0] Decode batch. #running-req: 3, #token: 51, token usage: 0.00, gen throughput (token/s): 201.91, #queue-req: 0


[2024-11-03 05:06:25 TP0] Decode batch. #running-req: 3, #token: 171, token usage: 0.00, gen throughput (token/s): 414.48, #queue-req: 0


[2024-11-03 05:06:26 TP0] Decode batch. #running-req: 3, #token: 291, token usage: 0.00, gen throughput (token/s): 411.44, #queue-req: 0


### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print_highlight("\n=== Testing asynchronous streaming generation ===")


async def main():
    for prompt in prompts:
        print_highlight(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        generator = await llm.async_generate(prompt, sampling_params, stream=True)
        async for chunk in generator:
            print(chunk["text"], end="", flush=True)
        print()


asyncio.run(main())

Generated text: 

 María

 Eug

enia

 García

...

.

 It

 can

 help

[2024-11-03 05:06:29 TP0] Decode batch. #running-req: 1, #token: 117, token usage: 0.00, gen throughput (token/s): 139.10, #queue-req: 0


In [6]:
llm.shutdown()